### Train CIFAR classifier using ResNet from scratch

In [ ]:
import torch
import torchvision
import lightning
from lightning.pytorch.loggers import CSVLogger
from torchvision import datasets, transforms
from torch.utils.data import dataset, DataLoader
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from common_def import LightningModel, plot_metrics_from_csv_log
import torch.nn.functional as F

#### Load the CIFAR-10 dataset

In [ ]:
class CIFAR10DataModule(lightning.LightningDataModule):
    def __init__(self, data_dir='./dataset/cifar-10', batch_size=64):
        super().__init__()
        self.data_dir = data_dir
        self.batch_size = batch_size

    def prepare_data(self):
        transform = transforms.Compose([
            transforms.ToTensor(),
            transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
        ])
        self.train_dataset = datasets.CIFAR10(root=self.data_dir, train=True, transform=transform, download=True)
        self.test_dataset = datasets.CIFAR10(root=self.data_dir, train=False, transform=transform, download=True)

    def setup(self, stage=None):
        self.train_dataset, self.val_dataset = dataset.random_split(dataset=self.train_dataset, lengths=[45000, 5000])

    def train_dataloader(self):
        return DataLoader(dataset=self.train_dataset, batch_size=self.batch_size, shuffle=True)

    def val_dataloader(self):
        return DataLoader(dataset=self.val_dataset, batch_size=self.batch_size, shuffle=False)

    def test_dataloader(self):
        return DataLoader(dataset=self.test_dataset, batch_size=self.batch_size, shuffle=False)
    

In [ ]:
cifar_dm = CIFAR10DataModule()
cifar_dm.prepare_data()
cifar_dm.setup()

#### Examine the dataset

Check dataset size

In [ ]:
print(f'train size {len(cifar_dm.train_dataset)}')
print(f'val size {len(cifar_dm.val_dataset)}')
print(f'test size {len(cifar_dm.test_dataset)}')

for image, label in cifar_dm.train_dataset:
    print(image)
    print(label)
    break 

Visualize an example

In [ ]:
for images, labels in cifar_dm.train_dataloader():
    break 

plt.figure(figsize=(8,8))
plt.axis('off')
plt.title('Training Images')
plt.imshow(
    np.transpose(torchvision.utils.make_grid(
        images,
        padding=1,
        pad_value=1.0,
        normalize=True
    ),
    (1, 2, 0)))

plt.show()

Labels
- 0: airplane
- 1: automobile
- 2: bird
- 3: cat
- 4: deer
- 5: dog
- 6: frog
- 7: horse
- 8: ship
- 9: truck

Check label distribution

In [ ]:
from collections import Counter

train_counter = Counter()
for images, labels in cifar_dm.train_dataloader():
    train_counter.update(labels.tolist())
print(f'train labels distribution: {sorted(train_counter.items())}')

val_counter = Counter()
for images, labels in cifar_dm.val_dataloader():
    val_counter.update(labels.tolist())
print(f'val labels distribution: {sorted(val_counter.items())}')

test_counter = Counter()
for images, labels in cifar_dm.test_dataloader():
    test_counter.update(labels.tolist())
print(f'test labels distribution: {sorted(test_counter.items())}')

#### Zero-rule classifier

In [ ]:
zerorule_acc = train_counter.most_common(1)[0][1] / sum(train_counter.values())
print(f'zero-rule accuracy: {zerorule_acc*100:.2f}%')

#### Define ResNet model

In [ ]:
class BasicBlock(torch.torch.nn.Module):
    expansion = 1

    def __init__(self, inplanes, planes, stride=1, downsample=None):
        super(BasicBlock, self).__init__()
        self.conv1 = torch.nn.Conv2d(inplanes, planes, kernel_size=3, stride=stride, padding=1, bias=False)
        self.bn1 = torch.nn.BatchNorm2d(planes)
        self.relu = torch.nn.ReLU(inplace=True)
        self.conv2 = torch.nn.Conv2d(planes, planes, kernel_size=3, stride=stride, padding=1, bias=False)
        self.bn2 = torch.nn.BatchNorm2d(planes)
        self.downsample = downsample
        self.stride = stride

    def forward(self, x):
        residual = x

        out = self.conv1(x)
        out = self.bn1(out)
        out = self.relu(out)

        out = self.conv2(out)
        out = self.bn2(out)

        if self.downsample is not None:
            residual = self.downsample(x)

        out += residual
        out = self.relu(out)

        return out

In [ ]:
class PyTorchResNet18(torch.nn.Module):

    def __init__(self, block, layers, num_classes, grayscale):
        super().__init__()
        
        if grayscale:
            in_dim = 1
        else:
            in_dim = 3
        self.inplanes = 64
        self.conv1 = torch.nn.Conv2d(in_dim, 64, kernel_size=7, stride=2, padding=3, bias=False)
        self.bn1 = torch.nn.BatchNorm2d(64)
        self.relu = torch.nn.ReLU(inplace=True)
        self.maxpool = torch.nn.MaxPool2d(kernel_size=3, stride=2, padding=1)
        self.layer1 = self._make_layer(block, 64, layers[0])
        self.layer2 = self._make_layer(block, 128, layers[1], stride=2)
        self.layer3 = self._make_layer(block, 256, layers[2], stride=2)
        self.layer4 = self._make_layer(block, 512, layers[3], stride=2)
        self.avgpool = torch.nn.AdaptiveAvgPool2d((1, 1))
        self.fc = torch.nn.Linear(512 * block.expansion, num_classes)

        for m in self.modules():
            if isinstance(m, torch.nn.Conv2d):
                n = m.kernel_size[0] * m.kernel_size[1] * m.out_channels
                m.weight.data.normal_(0, (2. / n)**.5)
            elif isinstance(m, torch.nn.BatchNorm2d):
                m.weight.data.fill_(1)
                m.bias.data.zero_()

    def _make_layer(self, block, planes, blocks, stride=1):
        downsample = None
        if stride != 1 or self.inplanes != planes * block.expansion:
            downsample = torch.nn.Sequential(
                torch.nn.Conv2d(self.inplanes, planes * block.expansion, kernel_size=1, stride=stride, bias=False),
                torch.nn.BatchNorm2d(planes * block.expansion),
            )

        layers = []
        layers.append(block(self.inplanes, planes, stride, downsample))
        self.inplanes = planes * block.expansion
        for i in range(1, blocks):
            layers.append(block(self.inplanes, planes))

        return torch.nn.Sequential(*layers)

    def forward(self, x):
        x = self.conv1(x)
        x = self.bn1(x)
        x = self.relu(x)
        x = self.maxpool(x)

        x = self.layer1(x)
        x = self.layer2(x)
        x = self.layer3(x)
        x = self.layer4(x)

        x = self.avgpool(x)
        x = torch.flatten(x, start_dim=1)
        logits = self.fc(x)
        probas = F.softmax(logits, dim=1)
        return logits

In [ ]:

def make_pytorch_resnet18(num_classes, grayscale=False):
    """Constructs a ResNet-18 model."""
    model = PyTorchResNet18(block=BasicBlock, 
                            layers=[2, 2, 2, 2],
                            num_classes=num_classes,
                            grayscale=grayscale)
    return model

#### Training

In [ ]:
lightning.seed_everything(123)

cifar_dm = CIFAR10DataModule()

torch_model = make_pytorch_resnet18(num_classes=10)
lightning_model = LightningModel(torch_model=torch_model, num_classes=10, learning_rate=0.1)

trainer = lightning.Trainer(
    max_epochs=10, 
    logger=CSVLogger('lightning_logs', name='ResNet18-CIFAR10'), 
    deterministic=True,
    accelerator='gpu', # cannot train on CPU
    devices=1
    )

trainer.fit(lightning_model, cifar_dm)